## Sprint 4
El siguiente trabajo es una continuación del sprint 3. El trabajo se adjunta en el repositorio donde se encuentra este notebook.

### Objetivo proyecto 4
- Agregar nuevas preguntas al analisis.
- Probar un modelo nuevo.

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import gc
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from scipy import sparse

In [111]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 941E-5588

 Directory of C:\Users\Tati\source\repos\acamica-ds-spr4

16/08/2021  14:40    <DIR>          .
16/08/2021  14:40    <DIR>          ..
22/07/2021  14:26                63 .gitignore
16/08/2021  13:47    <DIR>          .ipynb_checkpoints
16/08/2021  14:40           118,440 Reccomendation_System.ipynb
               2 File(s)        118,503 bytes
               3 Dir(s)  731,450,445,824 bytes free


Teniendo el trabajo anterior como referencia, se realiza un analisis exploratorio intentando validar algunas de las hipotesis que se plantean.

### Analisis exploratorio

Para comenzar el analisis utilizo una nueva fuente de datos diferente. En este caso partimos del dataset que obtenermos como output en el tp3.

In [112]:
data_reviews= pd.read_csv('C:\\Users\\Tati\\source\\repos\\data_full.csv')

In [113]:
import gzip
import pandas as pd

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)
        
data_games = []
for l in parse('C:\\Users\\Tati\\source\\repos\\acamica-datasets\steam_games.json.gz'):
        data_games.append(l)

In [114]:
data_games = pd.DataFrame(data_games)
data_games.head(4)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN


### DESCRIPCIÓN DE FEATURES DATA_REVIEWS
username: Nombre del usuario que ha dejado la reseña

hours: Cantidad de horas jugadas (acumuladas) en el juego al cual se le ha hecho la reseña

products: Cantidad de otros juegos probados

product_id: Identificacion númerica del juego reseñado

page_order: Número de página en el cual se encuentra el juego dentro de la plataforma

date: Fecha en la cual se ha creado la reseña

text: Comentario sobre el juego

early_access: Si es True significa que el juego puede ser adquirido durante su etapa de desarrollo

page: No es interpretable

user_id: Identificacion númerica del usuario creador de la reseña

found_funny: Cantidad de calificaciones positivas creadas por otros usuarios

compensation: Variable no interpretable dada la cantidad de valores nulos contenidos en ella

In [115]:
data_reviews.head()

,Unnamed: 0,Unnamed: 0.1,index,username,hours,products,product_id,text,Polarity,Calif,URL_NAME
0,0,0,13,MR_SERENITY,14.8,147.0,35140,aweosme game great story and some epic moments...,0.375000,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
1,1,1,15,Duha Nubie,11.3,73.0,35140,first debut feel sweet,0.300000,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
2,2,2,16,The Undead StalkeR,17.0,87.0,35140,9/10\nEven after all these years. this game re...,0.471429,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
3,3,3,32,Mono,14.1,54.0,506510,A Charming game with colourful pixel graphics ...,0.300000,3,Shadows_of_Adam
4,4,4,39,xiodeni,18.4,18.0,35140,I definitely recommend this game. I absolutely...,0.033333,1,Batman_Arkham_Asylum_Game_of_the_Year_Edition


### DESCRIPCIÓN DE FEATURES INCLUIDOS EN DATA_GAMES
desarrollador: Creador del juego

genres: Géneros, caracteristicas del juego

app_name: Editor del juego

title: Nombre del juego

url: Dirección web donde encontrarlo

release_date: Fecha de lanzamiento

tags: Etiquetas relacionadas con el juego

discount_price: Precio con descuento incluido

reviews_url: Dirección web donde encontrar las reseñas sobre el juego

specs: Especificaciones

price: Precio sin descuento

early_access: Si es True significa que el juego puede ser adquirido durante su etapa de desarrollo

id: Identificacion númerica del juego publicado

sentiment: Emoción promedio generada en los usuarios

metascore: Puntaje promedio recibido por los usuarios

In [116]:
data_games.head()

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


### Analisis exploratorio DATA_REVIEWS

Vemos cantidad de filas, columns e informacion sobre cada feature

In [117]:
data_reviews.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)

In [118]:
data_reviews.describe().applymap('{:,.2f}'.format)

,index,hours,products,product_id,Polarity,Calif
count,"2,313,124.00","2,313,124.00","2,313,124.00","2,313,124.00","2,313,124.00","2,313,124.00"
mean,"3,951,432.50",35.51,112.70,"244,474.73",0.12,1.89
std,"2,293,830.38",23.26,74.22,"143,103.28",0.27,0.98
min,13.00,10.10,16.00,10.00,-1.00,1.00
25%,"1,932,826.00",16.60,51.00,"203,290.00",0.00,1.00
50%,"3,934,443.00",27.70,95.00,"245,490.00",0.10,1.00
75%,"6,046,236.00",49.00,162.00,"330,840.00",0.25,3.00
max,"7,793,066.00",99.90,299.00,"773,900.00",1.00,3.00


In [119]:
data_reviews.shape

(2313124, 9)

Para ampliar el analisis anterior agregamos algunas preguntas:

#### ¿Cual fue el usuario que mas horas jugó?

In [120]:
data_reviews.loc[:,['hours','username']][data_reviews.hours == data_reviews.hours.max()]

,hours,username
266,99.9,assisterer
1925,99.9,Heroin™ |
11890,99.9,TenneyBox
14330,99.9,4d69636861656c
17812,99.9,FaStYI csgobounty.com
...,...,...
2311876,99.9,Joemo
2312331,99.9,Benny
2312686,99.9,CAP1stAVENGER
2312708,99.9,CAP1stAVENGER


#### ¿Cual fue el usuario que menos horas jugó?

In [121]:
data_reviews.loc[:,['hours','username']][data_reviews.hours == data_reviews.hours.min()]

,hours,username
85,10.1,ChivalrousWooly
206,10.1,Octopus Salad
248,10.1,UsamaJawad96
281,10.1,hindel1313
688,10.1,Vector Chromatron
...,...,...
2307985,10.1,Big Loaf
2310518,10.1,Yes Papa
2310537,10.1,Yes Papa
2310747,10.1,ZefSide HiTECH


Los limites son 10.1 y 99.9 porque previamente fijamos esos limites.

El minimo estara determinado por el minimo de horas que consideremos aceptables.

#### Que publisher hicieron mas juegos?

In [122]:
data_games["publisher"].value_counts().sort_values(ascending=False)

Ubisoft                      385
Dovetail Games - Trains      360
Degica                       286
Paradox Interactive          271
SEGA                         251
                            ... 
Elcaelum                       1
Silent Legends                 1
Remedy Entertainment Ltd.      1
Nikolai Usachev                1
Smashball Labs LLC             1
Name: publisher, Length: 8239, dtype: int64

In [123]:
data_reviews.reset_index(drop= True, inplace= True)
data_reviews

,index,username,hours,products,product_id,text,Polarity,Calif,URL_NAME
0,13,MR_SERENITY,14.8,147.0,35140,aweosme game great story and some epic moments...,0.375000,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
1,15,Duha Nubie,11.3,73.0,35140,first debut feel sweet,0.300000,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
2,16,The Undead StalkeR,17.0,87.0,35140,9/10\nEven after all these years. this game re...,0.471429,3,Batman_Arkham_Asylum_Game_of_the_Year_Edition
3,32,Mono,14.1,54.0,506510,A Charming game with colourful pixel graphics ...,0.300000,3,Shadows_of_Adam
4,39,xiodeni,18.4,18.0,35140,I definitely recommend this game. I absolutely...,0.033333,1,Batman_Arkham_Asylum_Game_of_the_Year_Edition
...,...,...,...,...,...,...,...,...,...
2313119,7793031,LostWhispers,92.1,150.0,252490,WARNING!! you will burn way to many hours in ...,0.050000,1,Rust
2313120,7793032,Rick Sanchez C-132,63.0,37.0,252490,This game is already one of the most fun and i...,0.108571,1,Rust
2313121,7793049,Crit_Magnet_,16.0,192.0,252490,"So much fun, once you get set up!",0.375000,3,Rust
2313122,7793061,worldoftanks,82.2,91.0,252490,O yes good waiting long time,0.325000,3,Rust


#### Vemos el promedio de cada juego.

In [124]:
dr1 = data_reviews.groupby("product_id").agg({"Calif": [np.mean, np.size, np.max]}).reset_index()
dr1

product_id     Calif          
                     mean size amax
0            10  1.707317   41    3
1            20  1.845745  376    3
2            30  2.031169  385    3
3            40  1.974359   39    3
4            50  1.961126  746    3
...         ...       ...  ...  ...
9733     761280  1.000000    1    1
9734     763410  3.000000    3    3
9735     763930  2.500000    4    3
9736     764240  1.000000    1    1
9737     773900  1.000000    1    1

[9738 rows x 4 columns]

Las preguntas respondida no arrojan ningun dato importante como para re-estructurar el dataset.

### Creamos un nuevo modelo utilizando la libreria LightFM

In [125]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

Se toman 500K juegos para realizar el analisis y entrenamiento del modelo.

Creamos una matriz e introducimos cero como valor en aquellos campos nullos (juegos que el usuario no jugo)

In [126]:
# Function to create an interaction matrix dataframe from transactional type interactions
data_reviews2 = data_reviews.head(10000)
#data_reviews2 = data_reviews 
interactions = data_reviews2.groupby(['username', 'product_id'])['Calif'].sum().unstack().reset_index().fillna(0).set_index('username')


In [127]:
interactions.shape

(9618, 148)

In [128]:
len(data_reviews2.product_id.unique())

148

In [129]:
del data_reviews

In [130]:
# Function to create a user dictionary based on their index and number in interaction dataset
username = list(interactions.index)
userdict = {}
counter = 0 
for i in username:
    userdict[i] = counter
    counter += 1

# Function to create an item dictionary based on their item_id and item name
games = data_games.reset_index()
item_dict ={}
for i in range(games.shape[0]):
    item_dict[(games.loc[i,'id'])] = games.loc[i,'title']

In [131]:
userdict

{'"Captain" Hair': 0,
 '"Lorience': 1,
 '"RiP Pc"': 2,
 '#"WGBS"-Danny': 3,
 '#DWB#Diamond Says': 4,
 '#NetNeutrality Bobo Butten': 5,
 '#TAF Ivanb006': 6,
 '#rekt': 7,
 '$Patrick-Dollar$': 8,
 '$[Cola-Freak]$': 9,
 '&^$##+GLITCH+##$^&': 10,
 "'   Pataum - Aquele das antigas": 11,
 "'eRt1 COOL VIBRATIONS": 12,
 '( + ) Kytic': 13,
 '( . Y . )': 14,
 '( ͡° ͜ʖ ͡°)': 15,
 '( ͡° ͜ʖ ͡°) Shiinaしい': 16,
 '(A13X)chicken general': 17,
 '(ISIG) JDEzekude': 18,
 '(Kuro) 1 Litre of Tears': 19,
 '(LT1) Cygnus': 20,
 '(RADIO): Need baka.': 21,
 '(SAY10) Pug | iz a pug ◐∩◑': 22,
 '(SCI) LG EmBeR': 23,
 '(´・ω・)っ由': 24,
 "(ง'̀-'́)ง": 25,
 '(ﾉ◕ヮ◕)ﾉ*:･ﾟ✧': 26,
 '*****': 27,
 "*Cthulhu R'lyeh*": 28,
 '*TLS*NightOwl': 29,
 '*Yuki*': 30,
 '*Zerm*': 31,
 ', the people,': 32,
 '- Athos -': 33,
 '- KureiKeii -': 34,
 '- Santo Lewis': 35,
 '- Zabijacka369Lv -': 36,
 '-13-': 37,
 '-=NIMH=-': 38,
 '-A': 39,
 '-Breeze-': 40,
 '-Ca!n-': 41,
 '-DI- Sairane': 42,
 '-FTL- Calico': 43,
 '-Fast~Slow II': 44,
 '-GK-Paco':

In [132]:
item_dict

{'761140': 'Lost Summoner Kitty',
 '643980': 'Ironbound',
 '670290': 'Real Pool 3D - Poolians',
 '767400': '弹炸人2222',
 '773570': nan,
 '772540': 'Battle Royale Trainer',
 '774276': 'SNOW - All Access Basic Pass',
 '774277': 'SNOW - All Access Pro Pass',
 '774278': 'SNOW - All Access Legend Pass',
 '768800': 'Race',
 '768570': 'Uncanny Islands',
 '724910': nan,
 '770380': 'Army of Tentacles: (Not) A Cthulhu Dating Sim: Black GOAT of the Woods Edition',
 '768880': 'Beach Rules',
 '765320': 'Planetarium 2 - Zen Odyssey',
 '773650': 'The first thrust of God',
 '774271': 'SNOW - Starter Pack',
 '773690': 'The first thrust of God - All Aircrafts',
 '774279': 'SNOW - Lifetime Pack',
 '772590': nan,
 '640250': nan,
 '768060': 'The Warrior Of Treasures',
 '711440': nan,
 '775880': 'RC Plane 3 - Canyon Scenario',
 '775881': 'RC Plane 3 - F-53B',
 '653960': 'DinoBlaster',
 '777910': 'Robotpencil Presents: Understanding 3D for Concept',
 '282010': 'Carmageddon Max Pack',
 '70': 'Half-Life',
 '7523

In [133]:
x = sparse.csr_matrix(interactions.values)

In [134]:
interactions.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Benchmark

In [141]:
model = LightFM(no_components= 300, loss='warp')

In [145]:
model.fit_partial(x,epochs=3000, num_threads = 4)

In [161]:
k=100
print('Train precision at k={}:\t{:.4f}'.format(k, precision_at_k(model, x, k=k).mean()))

Train precision at k=100:	0.0102


La precision del modelo no es demasiado alta, el puntaje es de 0.2043.

### Conclusion

Al igual que en el modelo anterior, la fuente de datos utilizada, al depender de una valor estimado por medio de un modelo, no parece confiable para entrenar otros algoritmos. 